In [13]:
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
import cv2
import os
import numpy as np
import random
import pickle
import time

加载所有图片数据，每个图片都resize成IMG_SIZE大小。整体打乱并将数据集写入文件，方便下次直接读取数据集而不是遍历文件夹一个一个的读取图片。

In [14]:
impath = ['datasets/English/Img/Sample%03d' % (x+1) for x in range(10)]
IMG_SIZE = 40
data_set = []
def load_data():
    for p in impath:
        label = impath.index(p)
        for pp in os.listdir(p):
            try:
                im = cv2.imread(os.path.join(p,pp),cv2.IMREAD_GRAYSCALE)
                new_im = cv2.resize(im,(IMG_SIZE,IMG_SIZE))
                data_set.append([new_im,label])
            except Exception as e:
                pass
            
load_data()
random.shuffle(data_set)
with open('data_set.pickle','wb') as f:
    pickle.dump(data_set,f)

将直接从文件中把数据集恢复。

In [15]:
def load_train_data():
    images = []
    labels = []
    with open('data_set.pickle','rb') as f:
        pickle_data = pickle.load(f)
    for x in pickle_data:
        images.append(x[0])
        labels.append(x[1])
    return images,labels
X_train,Y_train = load_train_data()
print(Y_train[:10])
#因为读取出来的数据是列表。所以需要用numpy把其reshape成numpy的数组
#X_train = np.reshape(X_train,(-1,IMG_SIZE,IMG_SIZE))
X_train = np.reshape(X_train,(-1,IMG_SIZE,IMG_SIZE,1))
Y_train = np.reshape(Y_train,(-1,))

[6, 3, 1, 1, 3, 2, 5, 8, 5, 9]


提取前十个数据验证一下是否正确

In [16]:
#for x in X_train[:3]:
#    plt.figure()
#    plt.imshow(x)
    
print(Y_train[:10])

[6 3 1 1 3 2 5 8 5 9]


将0-255的数据缩放到0-1之间

In [17]:
X_train = X_train / 255.0

定义模型

In [18]:
#配置tensorboard
filename = 'mymodel-{}.ckpt'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(filename))

model = Sequential()
model.add(Conv2D(128,(3,3),input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

#model.add(Dropout(0.2))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
#validation_split：测试数据和训练数据的分离比例。因为这两个其实是一样的数据，之前需要手动传入测试集，
#现在只需要定义一下比例，tensorflow自动会按比例随机选取测试样本
model.fit(X_train,Y_train,batch_size=10,validation_split=0.2,epochs=10,callbacks=[tensorboard])

Train on 440 samples, validate on 110 samples
Epoch 1/10
440/440 [==============================] - 6s 14ms/step - loss: 2.3074 - acc: 0.1364 - val_loss: 2.2069 - val_acc: 0.2364
Epoch 2/10
440/440 [==============================] - 6s 13ms/step - loss: 1.9045 - acc: 0.3568 - val_loss: 1.5990 - val_acc: 0.5182
Epoch 3/10
440/440 [==============================] - 6s 13ms/step - loss: 1.2251 - acc: 0.6136 - val_loss: 0.9922 - val_acc: 0.7000
Epoch 4/10
440/440 [==============================] - 6s 13ms/step - loss: 0.8894 - acc: 0.7182 - val_loss: 0.8486 - val_acc: 0.7455
Epoch 5/10
440/440 [==============================] - 6s 14ms/step - loss: 0.5856 - acc: 0.8341 - val_loss: 0.7758 - val_acc: 0.7545
Epoch 6/10
440/440 [==============================] - 6s 13ms/step - loss: 0.3714 - acc: 0.8818 - val_loss: 0.7508 - val_acc: 0.7727
Epoch 7/10
440/440 [==============================] - 6s 13ms/step - loss: 0.2670 - acc: 0.9295 - val_loss: 0.7335 - val_acc: 0.7636
Epoch 8/10
440/440 [===

测试前十个

In [21]:
pred = model.predict(X_train[:10])
for p in pred:
    print(np.argmax(p))

6
3
1
1
3
2
5
8
5
9


In [22]:
model.save('CNN-128-64-64-with-98-acc.model')